In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20191201)
tf.set_random_seed(20191201)

In [0]:
mnist = input_data.read_data_sets("./data/", one_hot=True)

In [0]:
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

keep_prob = tf.placeholder(tf.float32)

# Hidden Layer 1
num_filters1 = 32
W_conv1 = tf.Variable(tf.truncated_normal([3,3,1,num_filters1], stddev=0.1))
h_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME')
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Hidden Layer 2
num_filters2 = 64
W_conv2 = tf.Variable(tf.truncated_normal([3,3,num_filters1,num_filters2], stddev=0.1))
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME')
b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(h_pool2, keep_prob=keep_prob)

# Hidden Layer 3
num_filters3 = 128
W_conv3 = tf.Variable(tf.truncated_normal([3,3,num_filters2,num_filters3], stddev=0.1))
h_conv3 = tf.nn.conv2d(L2, W_conv3, strides=[1,1,1,1], padding='SAME')
b_conv3 = tf.Variable(tf.constant(0.1, shape=[num_filters3]))
h_conv3_cutoff = tf.nn.relu(h_conv3 + b_conv3)
h_pool3 = tf.nn.max_pool(h_conv3_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# Hidden Layer 4
num_filters4 = 256
W_conv4 = tf.Variable(tf.truncated_normal([3,3,num_filters3,num_filters4], stddev=0.1))
h_conv4 = tf.nn.conv2d(h_pool3, W_conv4, strides=[1,1,1,1], padding='SAME')
b_conv4 = tf.Variable(tf.constant(0.1, shape=[num_filters4]))
h_conv4_cutoff = tf.nn.relu(h_conv4 + b_conv4)
h_pool4 = tf.nn.max_pool(h_conv4_cutoff, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L4 = tf.nn.dropout(h_pool4, keep_prob=keep_prob)

In [0]:
# Fully Connected Layer
num_units1 = 2*2*num_filters4
num_units2 = 512

L4_flat = tf.reshape(L4, [-1, num_units1])

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
FC = tf.matmul(L4_flat, w2) + b2

# Output Layer
w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(FC, w0) + b0)

In [0]:
# Optimi
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for i in range(1, 2001):
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.7})
    if i % 100 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = int(len(mnist.test.labels) / 4 * c)
            end = int(len(mnist.test.labels) / 4 * (c+1))
            loss_val, acc_val = sess.run([loss, accuracy],
                                         feed_dict={x:mnist.test.images[start:end],
                                                    t:mnist.test.labels[start:end],
                                                    keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))

Step: 100, Loss: 3813.618164, Accuracy: 0.889700
Step: 200, Loss: 1340.638550, Accuracy: 0.957700
Step: 300, Loss: 1017.487305, Accuracy: 0.966500
Step: 400, Loss: 739.597290, Accuracy: 0.975900
Step: 500, Loss: 619.036133, Accuracy: 0.980000
Step: 600, Loss: 524.133423, Accuracy: 0.983300
Step: 700, Loss: 599.291321, Accuracy: 0.978600
Step: 800, Loss: 541.449219, Accuracy: 0.982000
Step: 900, Loss: 460.776764, Accuracy: 0.984600
Step: 1000, Loss: 414.296173, Accuracy: 0.986400
Step: 1100, Loss: 375.486572, Accuracy: 0.987000
Step: 1200, Loss: 467.724457, Accuracy: 0.983900
Step: 1300, Loss: 352.244751, Accuracy: 0.989100
Step: 1400, Loss: 333.113342, Accuracy: 0.988900
Step: 1500, Loss: 313.555817, Accuracy: 0.990000
Step: 1600, Loss: 404.409241, Accuracy: 0.987600
Step: 1700, Loss: 307.020447, Accuracy: 0.989600
Step: 1800, Loss: 329.876068, Accuracy: 0.988900
Step: 1900, Loss: 253.664368, Accuracy: 0.991300
Step: 2000, Loss: 248.288635, Accuracy: 0.991400


In [0]:
for i in range(2001, 4001):
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.7})
    if i % 100 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = int(len(mnist.test.labels) / 4 * c)
            end = int(len(mnist.test.labels) / 4 * (c+1))
            loss_val, acc_val = sess.run([loss, accuracy],
                                         feed_dict={x:mnist.test.images[start:end],
                                                    t:mnist.test.labels[start:end],
                                                    keep_prob:1.0})
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f' % (i, loss_val, acc_val))

Step: 2100, Loss: 361.571228, Accuracy: 0.987600
Step: 2200, Loss: 330.185303, Accuracy: 0.989500
Step: 2300, Loss: 315.612518, Accuracy: 0.989700
Step: 2400, Loss: 286.085724, Accuracy: 0.990600
Step: 2500, Loss: 376.625946, Accuracy: 0.988500
Step: 2600, Loss: 361.314728, Accuracy: 0.989400
Step: 2700, Loss: 308.100830, Accuracy: 0.990500
Step: 2800, Loss: 229.334976, Accuracy: 0.993200
Step: 2900, Loss: 234.117035, Accuracy: 0.992600
Step: 3000, Loss: 261.436188, Accuracy: 0.991700
Step: 3100, Loss: 257.223267, Accuracy: 0.991800
Step: 3200, Loss: 327.601837, Accuracy: 0.989900
Step: 3300, Loss: 288.860107, Accuracy: 0.991400
Step: 3400, Loss: 240.332794, Accuracy: 0.991800
Step: 3500, Loss: 235.577057, Accuracy: 0.991500
Step: 3600, Loss: 255.955811, Accuracy: 0.991200
Step: 3700, Loss: 257.720917, Accuracy: 0.990800
Step: 3800, Loss: 290.584351, Accuracy: 0.990500
Step: 3900, Loss: 225.680328, Accuracy: 0.993400
Step: 4000, Loss: 245.907135, Accuracy: 0.993100
